In [15]:
! pip install gymnasium[classic-control]
! mkdir logs
! mkdir plots

In [5]:
#import ale_py
#import shimmy
import gymnasium as gym
import torch
import json
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn import functional as F
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import random
import copy
import time
import pickle
import torchvision
import matplotlib.pyplot as plt
import math


In [6]:

class CNN(nn.Module):

    def __init__(self):
        super().__init__()   
        self.linear1 = nn.Linear(4, 128)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(128,128)
        self.activation = nn.ReLU()
        self.linear3 = nn.Linear(128,2)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        
        return x


In [18]:
class Memory():
    
    def __init__(self,size, prevent_forgetting=True):
        self.size = size
        self.experiences = []
        self.pop_index = 400 if prevent_forgetting else 0
        
    def sample(self,batch_size):
        return random.choices(self.experiences, k=batch_size)
        
    def add(self, experience):
        self.experiences.append(experience)
        if len(self.experiences) > self.size:
            self.experiences.pop(self.pop_index)
                    
    def __len__(self):
        return len(self.experiences)

In [14]:
class DQN_agent:


    def __init__(self, lr=0.0001 ,gamma=0.99, epsilon_params=(0.9,0.05,1000)):
        # Get cpu, gpu or mps device for training.
        self.device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        print(f"Using {self.device} device")
        self.pred_NN = CNN().to(self.device)
        self.target_NN = copy.deepcopy(self.pred_NN)
        self.target_NN.eval()
        self.gamma = gamma
        self.epsilon_start = epsilon_params[0]
        self.epsilon_end = epsilon_params[1]
        self.epsilon_decay = epsilon_params[2]
        self.optimizer = torch.optim.AdamW(self.pred_NN.parameters(), lr=lr)
        self.steps_done = 0
        
    def predict(self, x):
        self.steps_done += 1
        return self.pred_NN.forward(x)
    
    def action(self, pred):
        eps = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * math.exp(-1. * self.steps_done / self.epsilon_decay)
        return (
            random.randint(0, pred.size(dim=0) - 1)
            if random.random() < eps
            else torch.argmax(pred).item()
        )
    
    def train(self, experience_batch):
        loss_fn = nn.SmoothL1Loss()
        epoch_loss = 0
        states = torch.stack([experience_batch[i][0] for i in range(len(experience_batch))])
        actions = torch.tensor([experience_batch[i][1] for i in range(len(experience_batch))])
        rewards = torch.tensor([experience_batch[i][2] for i in range(len(experience_batch))])#torch.tensor(experience_batch[:][2])
        next_states = torch.stack([experience_batch[i][3] for i in range(len(experience_batch))])
        terminated = torch.tensor([not experience_batch[i][4] for i in range(len(experience_batch))])
        y = self.estimated_value( rewards, next_states, terminated)
        self.optimizer.zero_grad()
        pred = self.pred_NN(states)
        indicies = torch.LongTensor(actions)
        indicies =indicies.unsqueeze(dim=0).T
        pred = pred.gather(1,indicies)
        loss = loss_fn(y, pred)
        loss.backward()
        self.optimizer.step()
        epoch_loss += loss.item()
        return epoch_loss
        
    def copy(self):
        self.target_NN.load_state_dict(self.pred_NN.state_dict())  
        
    def estimated_value(self, reward, next_state, done):
        with torch.no_grad():# vectorize it
                target_pred = self.target_NN.forward(next_state)
                max_pred = torch.max(target_pred,1)[0].unsqueeze(1)
                done = done.unsqueeze(1)
                target = reward + self.gamma * torch.mul(max_pred,done)
        return target
        

In [19]:
#initialize environment
env = gym.make("CartPole-v1",render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, './videos', episode_trigger = lambda x: (x+1) % 100 == 0)
actions = range(env.action_space.n)

#hyperparams
max_steps = 9999
training_freq = 1
copying_freq = 40
batch_size = 128

#initialize agent
agent = DQN_agent(lr=0.0001)

training_session = 0
max_episode = 500

#loops until max_time is reached
memory = Memory(10000)
episodes = []
losses = []
total_steps = 0
logger = {"episodes":[],"losses":[]}
for episode  in range(1,max_episode):
    #get first states
    state, info = env.reset()
    #loops until experience_capacity is reached
    episode_reward = 0 
    episode_loss = 0
    steps = 0
    for i in range(1, max_steps):
        #predict q-values and choose action
        with torch.no_grad():
            pred = agent.predict(torch.tensor(state))
        action = agent.action(pred)
        #get next states
        next_state, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        if i == max_steps - 1:
            print("Max steps reached.")
        next_state = torch.from_numpy(next_state)
        experience = [torch.tensor(state), action, reward, torch.tensor(next_state), terminated] 
        memory.add(experience)
        if terminated or truncated:
            break
        state = next_state      
        if len(memory) > 500:
            experiences_train = memory.sample(batch_size)
            episode_loss += agent.train(experiences_train)
            training_session += 1
        steps = i
    if episode % 10 == 0:
        logger["episodes"].append(episodes)
        logger["losses"].append(losses)
        episodes = []
        losses = []
    episodes.append(episode_reward)
    losses.append(episode_loss/steps)
    if episode % copying_freq == 0:
        agent.copy()
    print(f"Episode: {episode} Reward: {episode_reward} loss: {episode_loss/steps}")
with open('logs/logger_dqn.json', 'w') as fp:
    json.dump(logger, fp)

Using cpu device
Episode: 1 Reward: 12.0 loss: 0.0
Episode: 2 Reward: 15.0 loss: 0.0
Episode: 3 Reward: 16.0 loss: 0.0
Episode: 4 Reward: 15.0 loss: 0.0
Episode: 5 Reward: 13.0 loss: 0.0
Episode: 6 Reward: 16.0 loss: 0.0
Episode: 7 Reward: 17.0 loss: 0.0
Episode: 8 Reward: 24.0 loss: 0.0
Episode: 9 Reward: 11.0 loss: 0.0


/tmp/ipykernel_28147/2539787781.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  experience = [torch.tensor(state), action, reward, torch.tensor(next_state), terminated]
/tmp/ipykernel_28147/2539787781.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = agent.predict(torch.tensor(state))


Episode: 10 Reward: 26.0 loss: 0.0
Episode: 11 Reward: 27.0 loss: 0.0
Episode: 12 Reward: 32.0 loss: 0.0
Episode: 13 Reward: 14.0 loss: 0.0
Episode: 14 Reward: 25.0 loss: 0.0
Episode: 15 Reward: 17.0 loss: 0.0
Episode: 16 Reward: 11.0 loss: 0.0
Episode: 17 Reward: 18.0 loss: 0.0
Episode: 18 Reward: 17.0 loss: 0.0
Episode: 19 Reward: 15.0 loss: 0.0
Episode: 20 Reward: 26.0 loss: 0.0
Episode: 21 Reward: 25.0 loss: 0.0
Episode: 22 Reward: 12.0 loss: 0.0
Episode: 23 Reward: 17.0 loss: 0.0
Episode: 24 Reward: 21.0 loss: 0.0
Episode: 25 Reward: 13.0 loss: 0.0
Episode: 26 Reward: 12.0 loss: 0.0
Episode: 27 Reward: 17.0 loss: 0.0
Episode: 28 Reward: 10.0 loss: 0.0
Episode: 29 Reward: 16.0 loss: 0.2926291445891062
Episode: 30 Reward: 31.0 loss: 0.3772726128498713
Episode: 31 Reward: 29.0 loss: 0.2237214334309101
Episode: 32 Reward: 13.0 loss: 0.14023145412405333
Episode: 33 Reward: 13.0 loss: 0.09897909065087636
Episode: 34 Reward: 13.0 loss: 0.07063179897765319
Episode: 35 Reward: 13.0 loss: 0

Moviepy - Done !
Moviepy - video ready /home/konrad/Repos/NCML_project/videos/rl-video-episode-99.mp4
Episode: 100 Reward: 9.0 loss: 0.10450922138988972
Episode: 101 Reward: 10.0 loss: 0.12462501724561055
Episode: 102 Reward: 10.0 loss: 0.10778834919134776
Episode: 103 Reward: 10.0 loss: 0.12257855633894603
Episode: 104 Reward: 10.0 loss: 0.13413669417301813
Episode: 105 Reward: 10.0 loss: 0.12207058568795522
Episode: 106 Reward: 10.0 loss: 0.09202131546205944
Episode: 107 Reward: 10.0 loss: 0.10618887841701508
Episode: 108 Reward: 10.0 loss: 0.10139347033368216
Episode: 109 Reward: 10.0 loss: 0.09931269536415736
Episode: 110 Reward: 9.0 loss: 0.1010338170453906
Episode: 111 Reward: 10.0 loss: 0.12371511343452665
Episode: 112 Reward: 10.0 loss: 0.1134082716372278
Episode: 113 Reward: 9.0 loss: 0.10743924137204885
Episode: 114 Reward: 10.0 loss: 0.1071991730067465
Episode: 115 Reward: 9.0 loss: 0.1276357425376773
Episode: 116 Reward: 10.0 loss: 0.12282569954792659
Episode: 117 Reward: 1

Moviepy - Done !
Moviepy - video ready /home/konrad/Repos/NCML_project/videos/rl-video-episode-199.mp4
Episode: 200 Reward: 10.0 loss: 0.19709087328778374
Episode: 201 Reward: 14.0 loss: 0.442082133430701


Episode: 202 Reward: 11.0 loss: 0.3182795614004135
Episode: 203 Reward: 13.0 loss: 0.2978632499774297
Episode: 204 Reward: 11.0 loss: 0.27280249148607255
Episode: 205 Reward: 13.0 loss: 0.2498964462429285
Episode: 206 Reward: 12.0 loss: 0.2292109734632752
Episode: 207 Reward: 14.0 loss: 0.2439601140526625
Episode: 208 Reward: 17.0 loss: 0.2583963042125106
Episode: 209 Reward: 12.0 loss: 0.22065636651082474
Episode: 210 Reward: 14.0 loss: 0.24003375264314505
Episode: 211 Reward: 17.0 loss: 0.27264812402427197
Episode: 212 Reward: 13.0 loss: 0.25688837592800456
Episode: 213 Reward: 16.0 loss: 0.256633914510409
Episode: 214 Reward: 14.0 loss: 0.2474339260504796
Episode: 215 Reward: 15.0 loss: 0.24529462627002172
Episode: 216 Reward: 13.0 loss: 0.2520565775533517
Episode: 217 Reward: 15.0 loss: 0.26328629893915995
Episode: 218 Reward: 13.0 loss: 0.22183513889710108
Episode: 219 Reward: 17.0 loss: 0.24825148470699787
Episode: 220 Reward: 12.0 loss: 0.25185973942279816
Episode: 221 Reward: 1

Moviepy - Done !
Moviepy - video ready /home/konrad/Repos/NCML_project/videos/rl-video-episode-299.mp4
Episode: 300 Reward: 17.0 loss: 0.35006748139858246
Episode: 301 Reward: 24.0 loss: 0.34171800315380096
Episode: 302 Reward: 22.0 loss: 0.3184684699489957
Episode: 303 Reward: 28.0 loss: 0.320419293862802
Episode: 304 Reward: 33.0 loss: 0.2873009261675179
Episode: 305 Reward: 22.0 loss: 0.3231289702511969
Episode: 306 Reward: 25.0 loss: 0.3043702443440755
Episode: 307 Reward: 18.0 loss: 0.3358861558577594
Episode: 308 Reward: 33.0 loss: 0.31894197035580873
Episode: 309 Reward: 27.0 loss: 0.3262368692801549
Episode: 310 Reward: 17.0 loss: 0.30800980841740966
Episode: 311 Reward: 65.0 loss: 0.29606536112260073
Episode: 312 Reward: 20.0 loss: 0.3133351418532823
Episode: 313 Reward: 71.0 loss: 0.3107104726135731
Episode: 314 Reward: 103.0 loss: 0.2985828178478222
Episode: 315 Reward: 90.0 loss: 0.2990307524967729
Episode: 316 Reward: 106.0 loss: 0.2838106057473591
Episode: 317 Reward: 97.

Moviepy - Done !
Moviepy - video ready /home/konrad/Repos/NCML_project/videos/rl-video-episode-399.mp4
Episode: 400 Reward: 122.0 loss: 0.09100282171544857
Episode: 401 Reward: 134.0 loss: 0.11450576439457841
Episode: 402 Reward: 148.0 loss: 0.09820488633407394
Episode: 403 Reward: 137.0 loss: 0.08617173886330634
Episode: 404 Reward: 161.0 loss: 0.07998197935812641
Episode: 405 Reward: 133.0 loss: 0.08902724629804266
Episode: 406 Reward: 135.0 loss: 0.10032973152849434
Episode: 407 Reward: 130.0 loss: 0.08692300377067091
Episode: 408 Reward: 148.0 loss: 0.0849997668081996
Episode: 409 Reward: 143.0 loss: 0.09419800127892505
Episode: 410 Reward: 172.0 loss: 0.07775873184402893
Episode: 411 Reward: 148.0 loss: 0.08000718745035532
Episode: 412 Reward: 119.0 loss: 0.08795700341135547
Episode: 413 Reward: 131.0 loss: 0.0782316374207417
Episode: 414 Reward: 144.0 loss: 0.08968580803684109
Episode: 415 Reward: 122.0 loss: 0.07949902628325234
Episode: 416 Reward: 141.0 loss: 0.0813136425517898